In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import numpy as np
import cv2
from PIL import Image

import cc3d

In [2]:
#値を-1から1に正規化する関数
def normalize_x(image):
    return image / 127.5 - 1


def denormalize_x(image):
    return (image + 1) * 127.5


#値を0から1正規化する関数
def normalize_y(image):
    return image / 255


#値を0から255に戻す関数
def denormalize_y(image):
    return image * 255

In [3]:
# インプット画像を読み込む関数
def load_X_gray(folder_path):
    
    image_files = []

    #image_files = os.listdir(folder_path)
       
    for file in os.listdir(folder_path):
        base, ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(file)
        else :
            pass
        
    image_files.sort()
    
    img = cv2.imread(folder_path + os.sep + image_files[0], cv2.IMREAD_GRAYSCALE)
    
    #image_files = image_files[1:]
    images = np.zeros((len(image_files), img.shape[0], img.shape[1], 1), np.float32)
    for i, image_file in enumerate(image_files):
        image = cv2.imread(folder_path + os.sep + image_file, cv2.IMREAD_GRAYSCALE)
        image = image[:, :, np.newaxis]
        images[i] = normalize_x(image)
    return images, image_files


def load_Y_gray_with_gaussian_and_numeric(folder_path, thresh = None , normalize = True, g_size = None):
    image_files = []
    #image_files = os.listdir(folder_path)
    
    for file in os.listdir(folder_path):
        base, ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(int(base))
        else:
            pass
        
    image_files.sort()
    
    img = cv2.imread(folder_path + os.sep + str(image_files[0]).zfill(4) + ".png", cv2.IMREAD_GRAYSCALE)
    
    images = np.zeros(
        (len(image_files), img.shape[0], img.shape[1], 1) ,np.float32
    )
    
    for i , image_file in enumerate(image_files):
        image = cv2.imread(
            folder_path + os.sep + str(image_file).zfill(4) + ".png" ,
            cv2.IMREAD_GRAYSCALE
        )
        #print(image.shape)
        
        # ぼかし処理
        if g_size:
            image = cv2.GaussianBlur(
                image, (g_size, g_size), 0
            )
        
        
        
        if thresh:
            ret , image = cv2.threshold(image , thresh , 255 , cv2.THRESH_BINARY)
        image = image[ : , : , np.newaxis]
        if normalize:
            images[i] = normalize_y(image)
        else:
            images[i] = image
    
    return images , image_files

In [4]:
def make_mask_img(ori_img, mask_img):
    mask_img_rgb = np.zeros((mask_img.shape[0], mask_img.shape[1], 3), np.float32)
    mask_img_rgb[:,:,0] = mask_img[:,:,0]
    mask_img_rgb[:,:,1] = mask_img[:,:,0]
    masked_img = cv2.addWeighted(mask_img_rgb,0.5,cv2.cvtColor(ori_img+0.75, cv2.COLOR_GRAY2BGR),0.6,0)
    return masked_img

In [5]:
def make_2_mask_img(ori_img, mask_img_1, mask_img_2):
    
    mask_img_rgb = np.zeros((mask_img_1.shape[0], mask_img_1.shape[1], 3), np.float32)
    mask_img_rgb[:,:,0] = mask_img_1[:,:,0]
    mask_img_rgb[:,:,1] = mask_img_1[:,:,0] + mask_img_2[:,:,0]
    mask_img_rgb[:,:,2] = mask_img_2[:,:,0]
    masked_img = cv2.addWeighted(mask_img_rgb,0.5,cv2.cvtColor(ori_img+0.75, cv2.COLOR_GRAY2BGR),0.6,0)
    return masked_img

In [6]:
def make_mask_img(ori_img, mask_img, mode="DL"):
    mask_img_rgb = np.zeros((mask_img.shape[0], mask_img.shape[1], 3), np.float32)
    if mode=="DL":
        mask_img_rgb[:,:,2] = mask_img[:,:,0]
    else:
        mask_img_rgb[:,:,1] = mask_img[:,:,0]
    masked_img = cv2.addWeighted(mask_img_rgb,0.5,cv2.cvtColor(ori_img+0.75, cv2.COLOR_GRAY2BGR),0.6,0)
    return masked_img

In [7]:
ori_imgs, _ = load_X_gray("I:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/superhuman_dataset/cropped_001/original/")

suga_imgs, _ = load_Y_gray_with_gaussian_and_numeric("I:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/superhuman_dataset/cropped_001/suga_seg/",  normalize=False)
nakamura_imgs, _ = load_Y_gray_with_gaussian_and_numeric("Z:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/superhuman_dataset/cropped_001/nakamura_seg//", normalize=False)

lamellar_imgs, _ = load_Y_gray_with_gaussian_and_numeric("I:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/superhuman_dataset/cropped_001/lamellar//", normalize=False)
tubular_imgs, _ = load_Y_gray_with_gaussian_and_numeric("I:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/superhuman_dataset/cropped_001/tubular//", normalize=False)


In [8]:
suga_lamellar_imgs = np.where(
    suga_imgs  ==1,
    255,
    0
)

suga_tubular_imgs = np.where(
    suga_imgs  ==2,
    255,
    0
)

nakamura_lamellar_imgs = np.where(
    nakamura_imgs  ==1,
    255,
    0
)

nakamura_tubular_imgs = np.where(
    nakamura_imgs  ==2,
    255,
    0
)

_lamellar_imgs = np.where(
    lamellar_imgs * 255 > 0,
    255,
    0
)

_tubular_imgs = np.where(
    tubular_imgs * 255 > 0,
    255,
    0
)

In [9]:
target  = nakamura_tubular_imgs.transpose(1, 2, 0, 3)[20]
label = cc3d.connected_components(target, connectivity=6)

In [10]:
new_target = np.zeros(label.shape)

for i in range(1, np.max(label)+1):
    print(np.sum(label==i))
    if np.sum(label==i) > 10:
        new_target += np.where(
            label == i,
            1,
            0
        )

7
21
42
17
3
6


In [11]:
out_dir = "Z:/paper_data/2021_cristae/for_revision_#2/fig4I"

os.makedirs(out_dir, exist_ok = True)

N = 20

masked_nakamura_imgs = make_mask_img(ori_imgs.transpose(1, 2, 0, 3)[N], new_target, mode="human")
masked_cristae_imgs = make_mask_img(ori_imgs.transpose(1, 2, 0, 3)[N], _tubular_imgs.transpose(1, 2, 0, 3)[N], mode="DL")


cv2.imwrite(
    f"{out_dir}/DL.png",
    denormalize_y(masked_cristae_imgs[50:105, 45:])
)

cv2.imwrite(
    f"{out_dir}/Human.png",
    denormalize_y(masked_nakamura_imgs[50:105, 45:])
)

True